In [2]:
!pip install -r requirements.txt

  Using cached aiohappyeyeballs-2.4.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiohttp-3.10.5-cp312-cp312-win_amd64.whl.metadata (7.8 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached dirtyjson-1.0.8-py3-none-any.whl.metadata (11 kB)
  Using cached distro-1.9.0-

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [31 lines of output]
      Traceback (most recent call last):
        File "c:\Deammy\Deam\1-Work\indiv_llm\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Deammy\Deam\1-Work\indiv_llm\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Deammy\Deam\1-Work\indiv_llm\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\เธ\x8aเธขเธ\x9eเธฑเธ—เธ\x98เน\x8c\AppData\Local\Temp\pip-build-env-0oogyjku\overlay\Lib\site-packag

In [9]:
!pip install PyPDF2

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)


In [11]:
!pip install PyPDF2 sentencepiece sentence-transformers numpy pandas faiss-cpu python-dotenv

In [10]:
from together import Together
from PyPDF2 import PdfReader
from llama_index.core.node_parser import SentenceSplitter
from sentence_transformers import SentenceTransformer
import pandas as pd

# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import (
    Settings
)
# from llama_index.core.retrievers import QueryFusionRetriever
# from llama_index.core.memory import ChatMemoryBuffer
# from unicodedata import normalize
# from llama_index.vector_stores.faiss import FaissVectorStore
import faiss
import numpy as np
d = 1536  # dimension
import os
import json

KeyboardInterrupt: 

In [ ]:
from dotenv import load_dotenv


load_dotenv()  

# uri = os.environ.get("MONGO_DB")
# Create a new client and connect to the server
# client = MongoClient(uri, server_api=ServerApi('1'))

# def connect_mongo(uri):
#     try:
#         client = MongoClient(uri, server_api=ServerApi('1'))
#         print("Connect Success")
#         return client
#     except:
#         print("Connection Error")
#         return None

# connect_mongo(uri)
model = SentenceTransformer('all-MiniLM-L6-v2')

### MultiTurn RAG CLASS

<p>This Version is demo, you can adapt all of it but be caeful of credit will be used.</p>

----

<strong>All Function</strong>
 
``` def __init__ :```

<p>document_path : Path to document.</br>
        history : Keep chat history with list</br>
        context_length : Determine length of context that query from retriever.</p>
</br>


``` def initial_retriever :```

<p>initialize retriever for retriever document.</br></p>
</br>


``` def prepare_context :```

<p>get context from retriever</br></p>
</br>


``` def update_history :```

<p>store history chat to self.history</br></p>
</br>


``` def generate_response :```

<p>get response from llm model.</br></p>
</br>


``` def get_response :```

<p>get user input and prepare data, then get response from LLM.</br></p>
</br>



----

In [2]:
"""
    class : MultiturnRAG
        init :
            document_path : Path to document.
            history : Keep chat history with list
            context_length : Determine length of context that query from retriever.

        def initial_retriever:
            initialize retriever for retriever document.

        def prepare_context:
            get context from retriever

        def update_history:
            store history chat to self.history

        def generate_response:
            get response from llm model.

        def get_response:
            get user input and prepare data, then get response from LLM.
"""



class MultiturnRAG:
    def __init__(self,model, client, document_path: list, context_length : int = 20):
        self.document_path = document_path
        self.history = []
        self.context_length = context_length
        self.model = model
        self.client = client
        self.context = ""
        
    def initial_retriever(self):

        """Initila Retriever

        input: -      
        output: -
    
        """

        splitter = SentenceSplitter(chunk_size=500, chunk_overlap=100)
        
        text = ""
        all_segment = []
        for path in self.document_path:
            reader = PdfReader(path)
            text = ""
            for page in reader.pages:
                text += page.extract_text()
            segment = splitter.split_text(text)
            for seg in segment:
                all_segment.append(seg)

        df = pd.DataFrame(all_segment, columns = ["Text"])
        print("Pass : 1")
        df['Embedding'] = df['Text'].apply(model.encode)
        vector = model.encode(df['Text'])
        dim = vector.shape[1]
        index = faiss.IndexFlatL2(dim)
        index.add(vector)
        self.retriever = index
        self.df = df


    def prepare_context(self, query):

        """Prepare Context

        input:
        query : input text for query document in retriever.
        
        output:
        context : context from document that relate to query.
    
        """

        encode_pre = model.encode(query)
        svec = np.array(encode_pre).reshape(1,-1)
        print("Pass : 3")
        distance,pos = self.retriever.search(svec,k=2)

        return self.df.Text.iloc[pos[0]]
    
    def update_history(self, user_input, bot_response):

        """Update History

        input:
        user_input : input text from user.
        sys_response : response from llm.
        
        """
        

        self.history.append({"role": "user", "content": user_input})
        self.history.append({"role": "assistant", "content": bot_response})
        if(len(self.history) == 6):
            print("Rechat : ")
            chat = "\n".join([str(history) for history in self.history])
            qa_prompt_str = f"""
                Context information is below.
                ---------------------
                {self.context}
                ---------------------
                Conclude this conversation
                {chat}
            """

            
            message = [{"role": "system", "content": "You are a helpful assistant. You must conclude this conversation.", "context" : self.context}]
            for history in self.history:
                message.append(history)
            message.append({"role": "user", "content": qa_prompt_str})

            response = self.client.chat.completions.create(
                model=self.model,
                messages=message,
                max_tokens=128,
                temperature=0.7,
                top_p=0.7,
                top_k=50,
                repetition_penalty=1,
                stop=["[/INST]","</s>"],
            )
            print(dict(dict(dict(response)['choices'][0])['message'])['content'].split("im_end")[0].replace("im_start", "").replace("assistant","").replace("<","").replace(">","").replace("|",""))
            self.context = dict(dict(dict(response)['choices'][0])['message'])['content'].split("im_end")[0].replace("im_start", "").replace("assistant","").replace("<","").replace(">","").replace("|","")
            self.history = []

        
    def generate_response(self, input, query_context):

        """Generate response
        
        input :
        input : text that will send to llm.
        context : context document from retriever.
        
        output
        response : response from llm.
        """

        input = input.replace("\b", "")

        print(self.context)
        if(self.context == ""):
            message = [{"role": "system", "content": "You are a helpful assistant."}] #You communicate in Thai language.
        else:
            message = [{"role": "system", "content": f"You are a helpful assistant. The context is {self.context}"}] #You communicate in Thai language.

        for history in self.history:
            message.append(history)

        message.append({"role": "user", "content": input, "context" : query_context})

        response = self.client.chat.completions.create(
            model=self.model,
            messages=message,
            max_tokens=64,
            temperature=0.7,
            top_p=0.7,
            top_k=50,
            repetition_penalty=1,
            stop=["[/INST]","</s>"],
        )
        print("Get response")

        return dict(dict(dict(response)['choices'][0])['message'])['content'].split("im_end")[0].replace("im_start", "").replace("assistant","").replace("<","").replace(">","").replace("|","")
            
    def get_response(self, user_input):

        """Get response
        input : 
        user_input : input from user.

        output :
        response : responser from model.

        """
        try:
            response = self.generate_response(input = user_input)
        except:
            return "Error"
        
        self.update_history(user_input = user_input,bot_response = response)
        
        # response = self.retriever.chat(user_input)
        return response


ModuleNotFoundError: No module named 'together'

In [2]:
class MultiturnRAG:
    def __init__(self,model, client, document_path: list, context_length : int = 20):
        self.document_path = document_path
        self.history = []
        self.context_length = context_length
        self.model = model
        self.client = client

    def initial_retriever(self):

        all_doc = []
        for path in self.document_path:
            document = SimpleDirectoryReader(
                input_files=[path]
            ).load_data()
            index = VectorStoreIndex.from_documents(document)
            all_doc.append(index)

        self.retriever  = QueryFusionRetriever(all_doc, 
                                               similarity_top_k=2, 
                                               num_queries=4,  # set this to 1 to disable query generation 
                                               use_async=True, 
                                               verbose=True,)


    def prepare_context(self, query):

        context = self.retriever.retrieve(query, top_k=self.context_length)
        return context
    
    def update_history(self, user_input, bot_response):
        self.history.append({"role": "user", "content": user_input})
        self.history.append({"role": "assistant", "content": bot_response})
        
    def generate_response(self, input, context):
        message = [{"role": "system", "content": "You are a helpful assistant. You communicate in Thai language."}]

        for history in self.history:
            message.append(history)

        message.append({"role": "user", "content": input})
        print(message)

        response = self.client.chat.completions.create(
            model=self.model,
            messages=message,
            max_tokens=64,
            temperature=0.7,
            top_p=0.7,
            top_k=50,
            repetition_penalty=1,
            stop=["[/INST]","</s>"],
        )
        print("Get response")

        return dict(dict(dict(response)['choices'][0])['message'])['content'].split("im_end")[0].replace("im_start", "").replace("assistant","").replace("<","").replace(">","").replace("|","")
            
    def get_response(self, user_input):
        response = self.generate_response(input = user_input, context="Normal Conversation")
        self.update_history(user_input = user_input,bot_response = response)
        return response


In [ ]:
document = ["./doc/attention.pdf"]
    model = "meta-llama/Meta-Llama-3-8B-Instruct-Lite"
    #Use for get api with llm
    client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))
    # Start RAG OOP
    rag = MultiturnRAG(document_path=document, model=model, client=client)
    Input = " "
    print("Start : \n")
    while(Input != "Exit"):
        Input = str(input())
        response = rag.get_response(Input)
        print(response)


In [11]:
!python -m ipykernel install --user --name=env_llama

Installed kernelspec env_llama in C:\Users\ชยพัทธ์\AppData\Roaming\jupyter\kernels\env_llama
